In [1]:
# Dependencies

import pandas as pd
from sqlalchemy import create_engine
from flask import Flask, jsonify
import json
import requests
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint



### Store CSV into DataFrame

In [2]:
csv_file = "Resources/DOHMH_New_York_City_Restaurant_Inspection_Results.csv"
inspection_data_df = pd.read_csv(csv_file)
inspection_data_df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50059672,GOOD FRIENDS 1,Brooklyn,1376,NOSTRAND AVE,11226.0,7182872345,Chinese,09/06/2018,Establishment Closed by DOHMH. Violations wer...,...,10/11/2019,Cycle Inspection / Initial Inspection,40.653158,-73.949837,317.0,40.0,82000.0,3116688.0,3.050850e+09,BK60
1,50034192,K'OOK,Manhattan,324,E 6TH ST,10003.0,2122540300,Korean,08/14/2017,Violations were cited in the following area(s).,...,10/11/2019,Cycle Inspection / Initial Inspection,40.727066,-73.987780,103.0,2.0,3800.0,1006234.0,1.004470e+09,MN22
2,50033885,A&H DELI,Manhattan,431,7TH AVE,10001.0,2125636200,American,06/06/2016,Violations were cited in the following area(s).,...,10/11/2019,Cycle Inspection / Re-inspection,40.750710,-73.990811,105.0,3.0,10100.0,1015218.0,1.008090e+09,MN17
3,41519373,BUNGALO,Queens,3203,BROADWAY,11106.0,7182047010,Armenian,01/21/2017,No violations were recorded at the time of thi...,...,10/11/2019,Inter-Agency Task Force / Initial Inspection,40.761538,-73.924450,401.0,22.0,6100.0,4008406.0,4.006140e+09,QN70
4,50016112,ANTOJITOS ECUATORIANOS,Brooklyn,3398,FULTON ST,11208.0,7182770970,"Latin (Cuban, Dominican, Puerto Rican, South &...",07/11/2018,Violations were cited in the following area(s).,...,10/11/2019,Cycle Inspection / Re-inspection,40.684208,-73.870173,305.0,37.0,118400.0,3092908.0,3.041490e+09,BK83


### Create new dataframe with select columns

In [3]:
# Select relevant columns: DBA (name), Building, Street, Zipcode, Boro, Grade

ny_inspect_df = inspection_data_df[['DBA', 'BUILDING', 'STREET', 'ZIPCODE', 'BORO', 'GRADE']].copy()
ny_inspect_df = ny_inspect_df.dropna()
ny_inspect_df.head()

,DBA,BUILDING,STREET,ZIPCODE,BORO,GRADE
2,A&H DELI,431,7TH AVE,10001.0,Manhattan,A
4,ANTOJITOS ECUATORIANOS,3398,FULTON ST,11208.0,Brooklyn,A
8,I LAND FISH & GRILL,7911,FLATLANDS AVE,11236.0,Brooklyn,B
9,CAFE LAFAYETTE,80,LAFAYETTE STREET,10013.0,Manhattan,A
12,FLY BAR,4224,COLLEGE POINT BLVD,11355.0,Queens,A


### Yelp API Response

In [4]:
#Setting up parameters for the API Key
api_key='KMXu7o4jj9H_5fBmmoxcQUXUcjaIiDMpnabg34SZhyJUQPt-H6y8sfBIq8jI65xOovUH7cKhDpTUnvK2UIFOf1r5864boyx0PCcIwR4QQ1OeR8IWr5RO7UxW3HJoXXYx'
headers = {'Authorization': 'Bearer %s' % api_key}

#Defining the Business Search end point url
url='https://api.yelp.com/v3/businesses/search'

In [5]:
# Pulling data for 1,000 restaurants in NYC
categories = ['Italian', 'American', 'Mexican', 'Chinese', 'Cuban']
restaurants = []

for category in categories:
    for offset in range(0,1000,50):
        params = {'term':'restaurants','categories': category,'location':'New York','limit':50, 'offset':offset}
        response = requests.get(url, params=params, headers=headers).json()
        restaurants.append(dict(response))

In [6]:
#pprint(restaurants[1])

In [27]:
#Defining empty dictionary to save only required elements
final = {}

#Defining Empty list to append all the data 
data = []

for restaurant in restaurants:
    try:
        for item in restaurant['businesses']:
            final['Restaurant Name'] = item['name']
            final['City'] = item['location']['city']
            final['Rating'] = item['rating']
            data.append(dict(final))
    except:   
        restaurant['businesses'] = float('nan')
        
        #Send api data to json file
        data.to_json("Resources/YelpData.json")

In [28]:
# Creating a DataFrame to save restaurant name, neighborhood, rating
json_file = "Resources/YelpData.json"
restaurants_df = pd.read_json(json_file)
restaurants_df.head()

,Restaurant Name,City,Rating
0,Upstate,New York,4.5
1,Amélie,New York,4.5
2,Clinton Street Baking Company,New York,4.0
3,LoveMama,New York,4.5
4,Barn Joo 35,New York,4.5


In [29]:
#Dropping Nan values
restaurants_df.dropna(axis='index',how='all',inplace=True)
restaurants_df.reset_index(drop=True)
restaurants_df.count()

Restaurant Name    5000
City               5000
Rating             5000
dtype: int64

In [30]:
# Convert restaurant names to uppercase to match DBA column in health inpection dataframe. This will be our primary key.
restaurants_df['Restaurant Name'] = restaurants_df['Restaurant Name'].str.upper() 
restaurants_df.head()

,Restaurant Name,City,Rating
0,UPSTATE,New York,4.5
1,AMÉLIE,New York,4.5
2,CLINTON STREET BAKING COMPANY,New York,4.0
3,LOVEMAMA,New York,4.5
4,BARN JOO 35,New York,4.5


### Transform Yelp_Data DataFrame

In [31]:
# Create a filtered dataframe from specific columns
restaurants_df_cols = ["Restaurant Name", "City", "Rating"]
restaurant_transformed= restaurants_df[restaurants_df_cols].copy()

# Rename the column headers
restaurant_transformed = restaurant_transformed.rename(columns={"Restaurant Name": "Restaurant_Name",
                                                          "City": "City",
                                                          "Rating": "Rating"})

# Clean the data by dropping duplicates and setting the index
restaurant_transformed.dropna(subset=['Restaurant_Name'],inplace=False)
restaurant_transformed.set_index("Restaurant_Name", inplace=False)

restaurant_transformed.head()

,Restaurant_Name,City,Rating
0,UPSTATE,New York,4.5
1,AMÉLIE,New York,4.5
2,CLINTON STREET BAKING COMPANY,New York,4.0
3,LOVEMAMA,New York,4.5
4,BARN JOO 35,New York,4.5


### Transform Ny_inspection Dataframe

In [32]:
# Create a filtered dataframe from specific columns
ny_inspect_cols = ["DBA", "BUILDING", "STREET", "ZIPCODE", "BORO", "GRADE"]
inspect_transformed= ny_inspect_df[ny_inspect_cols].copy()

# Rename the column headers
inspect_transformed = inspect_transformed.rename(columns={"DBA": "Restaurant_Name",
                                                          "BUILDING": "Building",
                                                          "STREET": "Street",
                                                          "ZIPCODE": "Zip_Code",
                                                          "BORO": "Boro",
                                                          "GRADE": "Grade"})

# Clean the data by dropping duplicates and setting the index
inspect_transformed.dropna(subset=['Restaurant_Name'], inplace=False)
inspect_transformed.set_index("Restaurant_Name", inplace=False)
inspect_transformed = inspect_transformed.reset_index(drop=True)

inspect_transformed.head()

,Restaurant_Name,Building,Street,Zip_Code,Boro,Grade
0,A&H DELI,431,7TH AVE,10001.0,Manhattan,A
1,ANTOJITOS ECUATORIANOS,3398,FULTON ST,11208.0,Brooklyn,A
2,I LAND FISH & GRILL,7911,FLATLANDS AVE,11236.0,Brooklyn,B
3,CAFE LAFAYETTE,80,LAFAYETTE STREET,10013.0,Manhattan,A
4,FLY BAR,4224,COLLEGE POINT BLVD,11355.0,Queens,A


### Connect to local database

In [33]:
rds_connection_string = "postgres:pace2013@localhost:5432/Yelp v. NYC"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [34]:
engine.table_names()

['Ny_Inspection', 'Yelp_Data']

### Load CSV converted DataFrame into database

In [40]:
inspect_transformed.to_sql(name='Ny_Inspection', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Restaurant_Name" of relation "Ny_Inspection" does not exist
LINE 1: INSERT INTO "Ny_Inspection" ("Restaurant_Name", "Building", ...
                                     ^

[SQL: INSERT INTO "Ny_Inspection" ("Restaurant_Name", "Building", "Street", "Zip_Code", "Boro", "Grade") VALUES (%(Restaurant_Name)s, %(Building)s, %(Street)s, %(Zip_Code)s, %(Boro)s, %(Grade)s)]
[parameters: ({'Restaurant_Name': 'A&H DELI', 'Building': '431', 'Street': '7TH AVE', 'Zip_Code': 10001.0, 'Boro': 'Manhattan', 'Grade': 'A'}, {'Restaurant_Name': 'ANTOJITOS ECUATORIANOS', 'Building': '3398', 'Street': 'FULTON ST', 'Zip_Code': 11208.0, 'Boro': 'Brooklyn', 'Grade': 'A'}, {'Restaurant_Name': 'I LAND FISH & GRILL', 'Building': '7911', 'Street': 'FLATLANDS AVE', 'Zip_Code': 11236.0, 'Boro': 'Brooklyn', 'Grade': 'B'}, {'Restaurant_Name': 'CAFE LAFAYETTE', 'Building': '80', 'Street': 'LAFAYETTE STREET', 'Zip_Code': 10013.0, 'Boro': 'Manhattan', 'Grade': '

### Load JSON converted DataFrame into database

In [42]:
restaurant_transformed.to_sql(name='Yelp_Data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "Yelp_Data" violates foreign key constraint "fk_Yelp_Data_Restaurant_Name"
DETAIL:  Key (Restaurant_Name)=(UPSTATE) is not present in table "Ny_Inspection".

[SQL: INSERT INTO "Yelp_Data" ("Restaurant_Name", "City", "Rating") VALUES (%(Restaurant_Name)s, %(City)s, %(Rating)s)]
[parameters: ({'Restaurant_Name': 'UPSTATE', 'City': 'New York', 'Rating': 4.5}, {'Restaurant_Name': 'AMÉLIE', 'City': 'New York', 'Rating': 4.5}, {'Restaurant_Name': 'CLINTON STREET BAKING COMPANY', 'City': 'New York', 'Rating': 4.0}, {'Restaurant_Name': 'LOVEMAMA', 'City': 'New York', 'Rating': 4.5}, {'Restaurant_Name': 'BARN JOO 35', 'City': 'New York', 'Rating': 4.5}, {'Restaurant_Name': 'THAI VILLA', 'City': 'New York', 'Rating': 4.5}, {'Restaurant_Name': 'BOCAPHE', 'City': 'New York', 'Rating': 4.0}, {'Restaurant_Name': 'ANYTIME', 'City': 'New York', 'Rating': 4.5}  ... displaying 10 of 5000 total bound parameter sets ...  {'Res

### Confirm data has been added

In [38]:
pd.read_sql_query('select * from "Ny_Inspection";', con=engine).head()

,DBA,BUILDING,STREET,ZIPCODE,BORO,GRADE


In [39]:
pd.read_sql_query('select * from "Yelp_Data";', con=engine).head()

,Restaurant_Name,City,Rating
